# Instructions

To run this notebook you will need to:
1. Download Ollam from [here](https://ollama.com/)
2. Go to your terminal and type: `ollama pull llama3` to download llama3 model in your local computer.

## Imports

In [41]:
import pandas as pd
import subprocess
import ast
import re

## Function to extract judge's name

In [42]:
def generate_judge_sentence(text, model="llama3"):
    """
    Generate a sentence identifying the judges in a given court text.

    Parameters
    ----------
    text : str
        The court text to analyze.
    model : str, optional
        The name of the language model to use (default is "llama3").

    Returns
    -------
    str
        A sentence either listing the judges or stating that none are mentioned.
    """
    prompt = f"""
You are a legal assistant. Your task is to identify the names of the judge(s) who presided over the case from the following court text.

Instructions:
- Return a single sentence that starts with "The judges in this case are ..." followed by the judge names.
- If no judge is mentioned, return "No judges are mentioned in the case."
- Do not assume any judges if it is not mentioned.
- Keep your answer in 1 sentence.

Court Text:
{text}
"""
    result = subprocess.run(
        ["ollama", "run", model],
        input=prompt.encode(),
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE
    )
    return result.stdout.decode().strip()


def extract_names_from_judge_sentence(sentence, model="llama3"):
    """
    Extract only the judge names from a sentence.

    Parameters
    ----------
    sentence : str
        A sentence that mentions the judges (e.g., output of `generate_judge_sentence`).
    model : str, optional
        The name of the language model to use (default is "llama3").

    Returns
    -------
    list of str
        A list of judge names with titles removed. Returns an empty list if no names are found.
    """
    prompt = f"""
You are a legal parser. Extract only the judge names from the sentence below.

Instructions:
- Return the names in a valid Python list of strings.
- Do not include any titles like "Judge", "Justice", or "Chief Justice".
- If no names are found, return: []

Sentence:
{sentence}

Output:
"""
    result = subprocess.run(
        ["ollama", "run", model],
        input=prompt.encode(),
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE
    )
    output = result.stdout.decode().strip()

    match = re.search(r"\[.*?\]", output, re.DOTALL)
    if match:
        try:
            return ast.literal_eval(match.group(0))
        except Exception:
            return []
    return []


def extract_judges_from_dataframe(df, text_column="unofficial_text", model="llama3"):
    """
    Extract judge names from a DataFrame containing court texts.

    Parameters
    ----------
    df : pandas.DataFrame
        The DataFrame containing court text data.
    text_column : str, optional
        The column name in `df` that contains the court text (default is "unofficial_text").
    model : str, optional
        The language model to use for generation and parsing (default is "llama3").

    Returns
    -------
    pandas.DataFrame
        A new DataFrame with an additional 'judges' column containing lists of judge names.
    """
    df = df.copy()
    df["judges"] = None

    for idx, row in df.iterrows():
        full_text = row[text_column]
        lines = full_text.splitlines()
        first_30 = "\n".join(lines[:30])

        judge_sentence = generate_judge_sentence(first_30, model=model)
        judge_names = extract_names_from_judge_sentence(judge_sentence, model=model)

        df.at[idx, "judges"] = judge_names
        print(f"Row {idx} processed.")
    
    return df

## Supreme Court Data

In [43]:
SCC = pd.read_csv("../data/processed/SCC_Regex.csv")
SCC

,Unnamed: 0,citation,dataset,year,language,document_date,source_url,unofficial_text,inadmissibility_reason
0,0,2014 SCC 37,SCC,2014,en,2014-05-14,https://decisions.scc-csc.ca/scc-csc/scc-csc/e...,Canada (Citizenship and Immigration) v. Harkat...,"['security', 'human_rights', 'serious_criminal..."
1,1,2014 SCC 68,SCC,2014,en,2014-10-30,https://decisions.scc-csc.ca/scc-csc/scc-csc/e...,Febles v. Canada (Citizenship and Immigration)...,['refugee']
2,2,2015 SCC 58,SCC,2015,en,2015-11-27,https://decisions.scc-csc.ca/scc-csc/scc-csc/e...,B010 v. Canada (Citizenship and Immigration)\n...,['refugee']
3,3,2015 SCC 61,SCC,2015,en,2015-12-10,https://decisions.scc-csc.ca/scc-csc/scc-csc/e...,Kanthasamy v. Canada (Citizenship and Immigrat...,['refugee']
4,4,2019 SCC 65,SCC,2019,en,2019-12-19,https://decisions.scc-csc.ca/scc-csc/scc-csc/e...,Canada (Minister of Citizenship and Immigratio...,"['security', 'human_rights', 'serious_criminal..."
5,5,2023 SCC 17,SCC,2023,en,2023-06-16,https://decisions.scc-csc.ca/scc-csc/scc-csc/e...,Canadian Council for Refugees v. Canada (Citiz...,['refugee']
6,6,2023 SCC 21,SCC,2023,en,2023-09-27,https://decisions.scc-csc.ca/scc-csc/scc-csc/e...,Mason v. Canada (Citizenship and Immigration)\...,['refugee']


In [ ]:
# SCC_final = SCC.query("~inadmissibility_reason.str.contains('other') and ~inadmissibility_reason.str.contains('refugee')", engine='python')\
#     .reset_index().drop(['index', 'Unnamed: 0'], axis = 1)
# SCC_final

,citation,dataset,year,language,document_date,source_url,unofficial_text,inadmissibility_reason
0,2014 SCC 37,SCC,2014,en,2014-05-14,https://decisions.scc-csc.ca/scc-csc/scc-csc/e...,Canada (Citizenship and Immigration) v. Harkat...,"['security', 'human_rights', 'serious_criminal..."
1,2019 SCC 65,SCC,2019,en,2019-12-19,https://decisions.scc-csc.ca/scc-csc/scc-csc/e...,Canada (Minister of Citizenship and Immigratio...,"['security', 'human_rights', 'serious_criminal..."


In [64]:
SSC_judges = extract_judges_from_dataframe(SCC, text_column="unofficial_text")

Row 0 processed.
Row 1 processed.
Row 2 processed.
Row 3 processed.
Row 4 processed.
Row 5 processed.
Row 6 processed.


In [65]:
SSC_judges = SSC_judges.drop('Unnamed: 0', axis = 1).reset_index()
SSC_judges = SSC_judges.drop('index', axis = 1)
SSC_judges

,citation,dataset,year,language,document_date,source_url,unofficial_text,inadmissibility_reason,judges
0,2014 SCC 37,SCC,2014,en,2014-05-14,https://decisions.scc-csc.ca/scc-csc/scc-csc/e...,Canada (Citizenship and Immigration) v. Harkat...,"['security', 'human_rights', 'serious_criminal...","[McLachlin, LeBel, Abella, Rothstein, Cromwell..."
1,2014 SCC 68,SCC,2014,en,2014-10-30,https://decisions.scc-csc.ca/scc-csc/scc-csc/e...,Febles v. Canada (Citizenship and Immigration)...,['refugee'],"[McLachlin, LeBel, Abella, Rothstein, Cromwell..."
2,2015 SCC 58,SCC,2015,en,2015-11-27,https://decisions.scc-csc.ca/scc-csc/scc-csc/e...,B010 v. Canada (Citizenship and Immigration)\n...,['refugee'],"[McLachlin, Abella, Rothstein, Moldaver, Karak..."
3,2015 SCC 61,SCC,2015,en,2015-12-10,https://decisions.scc-csc.ca/scc-csc/scc-csc/e...,Kanthasamy v. Canada (Citizenship and Immigrat...,['refugee'],"[McLachlin, Abella, Cromwell, Moldaver, Karaka..."
4,2019 SCC 65,SCC,2019,en,2019-12-19,https://decisions.scc-csc.ca/scc-csc/scc-csc/e...,Canada (Minister of Citizenship and Immigratio...,"['security', 'human_rights', 'serious_criminal...","[Wagner, Abella, Moldaver, Karakatsanis, Gasco..."
5,2023 SCC 17,SCC,2023,en,2023-06-16,https://decisions.scc-csc.ca/scc-csc/scc-csc/e...,Canadian Council for Refugees v. Canada (Citiz...,['refugee'],"[Wagner, Karakatsanis, Côte, Brown, Rowe, Mart..."
6,2023 SCC 21,SCC,2023,en,2023-09-27,https://decisions.scc-csc.ca/scc-csc/scc-csc/e...,Mason v. Canada (Citizenship and Immigration)\...,['refugee'],[]


In [74]:
SSC_judges.to_csv("../data/processed/SCC_judges.csv")

## Federal Court of Appeal

In [68]:
FCA = pd.read_csv("../data/processed/FCA_Regex.csv")
FCA

,Unnamed: 0,citation,dataset,year,language,document_date,source_url,unofficial_text,inadmissibility_reason
0,0,2014 FCA 113,FCA,2014,en,2014-05-02,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Kanthasamy v. Canada (Citizenship and Immigrat...,['refugee']
1,1,2014 FCA 114,FCA,2014,en,2014-05-02,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Lemus v. Canada (Citizenship and Immigration)\...,['refugee']
2,2,2014 FCA 126,FCA,2014,en,2014-05-14,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Kinsel v. Canada (Citizenship and Immigration)...,"['security', 'human_rights', 'serious_criminal..."
3,3,2014 FCA 157,FCA,2014,en,2014-06-10,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Sanchez v. Canada (Citizenship and Immigration...,"['serious_criminality', 'criminality']"
4,4,2014 FCA 160,FCA,2014,en,2014-06-12,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Canada (Citizenship and Immigration) v. Savin\...,['human_rights']
...,...,...,...,...,...,...,...,...,...
94,94,2023 FCA 36,FCA,2023,en,2023-02-16,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Lukács v. Canada (Citizenship and Immigration)...,['refugee']
95,95,2024 FCA 165,FCA,2024,en,2024-10-08,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Nanhar v. Canada (Citizenship and Immigration)...,['refugee']
96,96,2024 FCA 174,FCA,2024,en,2024-10-24,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Li v. Canada (Citizenship and Immigration)\nCo...,"['human_rights', 'serious_criminality', 'crimi..."
97,97,2020 CAF 126,FCA,2020,fr,2020-07-28,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Canada (Citoyenneté et Immigration) c. Solmaz\...,['refugee']


In [ ]:
# FCA_final = FCA.query("~inadmissibility_reason.str.contains('other') and ~inadmissibility_reason.str.contains('refugee')", engine='python')\
#      .reset_index().drop(['index', 'Unnamed: 0'], axis = 1)
# FCA_final

In [70]:
FCA_judges = extract_judges_from_dataframe(FCA, text_column="unofficial_text")

Row 0 processed.
Row 1 processed.
Row 2 processed.
Row 3 processed.
Row 4 processed.
Row 5 processed.
Row 6 processed.
Row 7 processed.
Row 8 processed.
Row 9 processed.
Row 10 processed.
Row 11 processed.
Row 12 processed.
Row 13 processed.
Row 14 processed.
Row 15 processed.
Row 16 processed.
Row 17 processed.
Row 18 processed.
Row 19 processed.
Row 20 processed.
Row 21 processed.
Row 22 processed.
Row 23 processed.
Row 24 processed.
Row 25 processed.
Row 26 processed.
Row 27 processed.
Row 28 processed.
Row 29 processed.
Row 30 processed.
Row 31 processed.
Row 32 processed.
Row 33 processed.
Row 34 processed.
Row 35 processed.
Row 36 processed.
Row 37 processed.
Row 38 processed.
Row 39 processed.
Row 40 processed.
Row 41 processed.
Row 42 processed.
Row 43 processed.
Row 44 processed.
Row 45 processed.
Row 46 processed.
Row 47 processed.
Row 48 processed.
Row 49 processed.
Row 50 processed.
Row 51 processed.
Row 52 processed.
Row 53 processed.
Row 54 processed.
Row 55 processed.
Ro

In [71]:
FCA_judges = FCA_judges.drop('Unnamed: 0', axis = 1).reset_index()
FCA_judges = FCA_judges.drop('index', axis = 1)
FCA_judges

,citation,dataset,year,language,document_date,source_url,unofficial_text,inadmissibility_reason,judges
0,2014 FCA 113,FCA,2014,en,2014-05-02,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Kanthasamy v. Canada (Citizenship and Immigrat...,['refugee'],"[BLAIS, SHARLOW, STRA TAS]"
1,2014 FCA 114,FCA,2014,en,2014-05-02,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Lemus v. Canada (Citizenship and Immigration)\...,['refugee'],"[BLAIS, SHARLOW, STRA TAS]"
2,2014 FCA 126,FCA,2014,en,2014-05-14,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Kinsel v. Canada (Citizenship and Immigration)...,"['security', 'human_rights', 'serious_criminal...","[Dawson, Trudel, Near]"
3,2014 FCA 157,FCA,2014,en,2014-06-10,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Sanchez v. Canada (Citizenship and Immigration...,"['serious_criminality', 'criminality']","[Nadon, Stratas, Scott]"
4,2014 FCA 160,FCA,2014,en,2014-06-12,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Canada (Citizenship and Immigration) v. Savin\...,['human_rights'],"[Gauthier, Mainville, Boivin]"
...,...,...,...,...,...,...,...,...,...
94,2023 FCA 36,FCA,2023,en,2023-02-16,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Lukács v. Canada (Citizenship and Immigration)...,['refugee'],"[PELLETIER, WOODS, ROUSSEL]"
95,2024 FCA 165,FCA,2024,en,2024-10-08,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Nanhar v. Canada (Citizenship and Immigration)...,['refugee'],"[Webb, Mactavish, Walker]"
96,2024 FCA 174,FCA,2024,en,2024-10-24,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Li v. Canada (Citizenship and Immigration)\nCo...,"['human_rights', 'serious_criminality', 'crimi...","[Stratas, Boivin, Heckman]"
97,2020 CAF 126,FCA,2020,fr,2020-07-28,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Canada (Citoyenneté et Immigration) c. Solmaz\...,['refugee'],"[Boivin, De Montigny, Leblanc]"


In [72]:
FCA_judges.to_csv("../data/processed/FCA_judges.csv")

# Federal Court

In [73]:
FC = pd.read_csv("../data/processed/FC_Regex.csv")
FC

,Unnamed: 0,citation,dataset,year,language,document_date,source_url,unofficial_text,inadmissibility_reason
0,0,2014 FC 10,FC,2014,en,2014-01-07,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,Singh v. Canada (Citizenship and Immigration)\...,['refugee']
1,1,2014 FC 1000,FC,2014,en,2014-10-21,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,Vinat v. Canada (Citizenship and Immigration)\...,['refugee']
2,2,2014 FC 1002,FC,2014,en,2014-10-23,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,Almrei v. Canada (Citizenship and Immigration)...,['refugee']
3,3,2014 FC 1003,FC,2014,en,2014-10-22,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,Avagyan v. Canada (Citizenship and Immigration...,['refugee']
4,4,2014 FC 1004,FC,2014,en,2014-10-22,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,Avagyan v. Canada (Citizenship and Immigration...,['refugee']
...,...,...,...,...,...,...,...,...,...
8688,8688,2024 CF 677,FC,2024,fr,2024-05-02,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,Betanzos Ramirez c. Canada (Citoyenneté et Imm...,['refugee']
8689,8689,2024 CF 74,FC,2024,fr,2024-01-18,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,Abdallah Elnour El Senoussi c. Canada (Citoyen...,['refugee']
8690,8690,2024 CF 79,FC,2024,fr,2024-01-18,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,Contreras Callado c. Canada (Citoyenneté et Im...,['refugee']
8691,8691,2024 CF 966,FC,2024,fr,2024-06-21,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,Blandon Quintero c. Canada (Citoyenneté et Imm...,['refugee']


In [ ]:
# FC_final = FC.query("~inadmissibility_reason.str.contains('other') and ~inadmissibility_reason.str.contains('refugee')", engine='python')\
#       .reset_index().drop(['index', 'Unnamed: 0'], axis = 1)
# FC_final

,citation,dataset,year,language,document_date,source_url,unofficial_text,inadmissibility_reason
11922,2014 FC 1,FC,2014,en,2014-01-02,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,Telus Communications Company v. Canada (Attorn...,"['human_rights', 'serious_criminality', 'crimi..."
11930,2014 FC 1008,FC,2014,en,2014-10-22,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,Bautista v. Canada (Citizenship and Immigratio...,['misrepresentation']
11933,2014 FC 1011,FC,2014,en,2014-10-23,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,Canada (Public Safety and Emergency Preparedne...,['non_compliance']
11934,2014 FC 1012,FC,2014,en,2014-10-23,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,Po v. Canada (Citizenship and Immigration)\nCo...,['human_rights']
11935,2014 FC 1015,FC,2014,en,2014-10-24,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,Gordon v. Canada (Citizenship and Immigration)...,"['human_rights', 'misrepresentation']"
...,...,...,...,...,...,...,...,...
23170,2022 CF 882,FC,2022,fr,2022-06-16,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,Jules c. Canada (Citoyenneté et Immigration)\n...,"['human_rights', 'serious_criminality', 'crimi..."
23173,2023 CF 1619,FC,2023,fr,2023-12-01,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,Salhi c. Canada (Citoyenneté et Immigration)\n...,"['serious_criminality', 'criminality']"
23177,2024 CF 151,FC,2024,fr,2024-01-30,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,Salazar Vargas c. Canada (Citoyenneté et Immig...,"['serious_criminality', 'criminality']"
23178,2024 CF 327,FC,2024,fr,2024-02-28,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,Rashidian c. Canada (Procureur général)\nBase ...,['human_rights']


In [ ]:
FC_judges = extract_judges_from_dataframe(FC, text_column="unofficial_text")

Row 11922 processed.
Row 11930 processed.
Row 11933 processed.
Row 11934 processed.
Row 11935 processed.
Row 11941 processed.
Row 11944 processed.
Row 11947 processed.
Row 11948 processed.
Row 11949 processed.
Row 11950 processed.
Row 11951 processed.
Row 11954 processed.
Row 11955 processed.
Row 11957 processed.
Row 11959 processed.
Row 11960 processed.
Row 11961 processed.
Row 11967 processed.
Row 11968 processed.
Row 11971 processed.
Row 11977 processed.
Row 11979 processed.
Row 11983 processed.
Row 11985 processed.
Row 11990 processed.
Row 11994 processed.
Row 11996 processed.
Row 11998 processed.
Row 12001 processed.
Row 12003 processed.
Row 12004 processed.
Row 12007 processed.
Row 12010 processed.
Row 12014 processed.
Row 12018 processed.
Row 12020 processed.
Row 12021 processed.
Row 12025 processed.
Row 12027 processed.
Row 12034 processed.
Row 12038 processed.
Row 12042 processed.
Row 12043 processed.
Row 12046 processed.
Row 12048 processed.
Row 12052 processed.
Row 12053 pro

In [ ]:
FC_judges = FC_judges.drop('Unnamed: 0', axis = 1).reset_index()
FC_judges = FC_judges.drop('index', axis = 1)
FC_judges

In [39]:
FC_judges.to_csv("../data/processed/FC_judges.csv")

In [40]:
FC_judges

,citation,dataset,year,language,document_date,source_url,unofficial_text,inadmissibility_reason,judges
11922,2014 FC 1,FC,2014,en,2014-01-02,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,Telus Communications Company v. Canada (Attorn...,"['human_rights', 'serious_criminality', 'crimi...",[Strickland]
11930,2014 FC 1008,FC,2014,en,2014-10-22,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,Bautista v. Canada (Citizenship and Immigratio...,['misrepresentation'],[Diner]
11933,2014 FC 1011,FC,2014,en,2014-10-23,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,Canada (Public Safety and Emergency Preparedne...,['non_compliance'],"[Shore, Gauthier]"
11934,2014 FC 1012,FC,2014,en,2014-10-23,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,Po v. Canada (Citizenship and Immigration)\nCo...,['human_rights'],[Tremblay-Lamer]
11935,2014 FC 1015,FC,2014,en,2014-10-24,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,Gordon v. Canada (Citizenship and Immigration)...,"['human_rights', 'misrepresentation']",[Locke]
...,...,...,...,...,...,...,...,...,...
23170,2022 CF 882,FC,2022,fr,2022-06-16,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,Jules c. Canada (Citoyenneté et Immigration)\n...,"['human_rights', 'serious_criminality', 'crimi...",[St-Louis]
23173,2023 CF 1619,FC,2023,fr,2023-12-01,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,Salhi c. Canada (Citoyenneté et Immigration)\n...,"['serious_criminality', 'criminality']",[Azmudeh]
23177,2024 CF 151,FC,2024,fr,2024-01-30,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,Salazar Vargas c. Canada (Citoyenneté et Immig...,"['serious_criminality', 'criminality']",[Heneghan]
23178,2024 CF 327,FC,2024,fr,2024-02-28,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,Rashidian c. Canada (Procureur général)\nBase ...,['human_rights'],"[Ngo, Tabib, Steele]"
